In [ ]:
import h5py
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [ ]:
tf.keras.utils.set_random_seed(42)

In [ ]:
with h5py.File('SVHN_single_grey1.h5', 'r') as file:
    x_train = np.array(file['X_train'])
    y_train = np.array(file['y_train'])
    x_test = np.array(file['X_test'])
    y_test = np.array(file['y_test'])
    x_val = np.array(file['X_val'])
    y_val = np.array(file['y_val'])

In [ ]:
x_train[0].shape

In [ ]:
y_train.shape

In [ ]:
# Load and preprocess the SVHN dataset
x_train, x_test, x_val = x_train / 255.0, x_test / 255.0, x_val / 255.0

In [ ]:
# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
# Constants
input_shape = x_train.shape[1:]
nClasses = 10

In [ ]:
def createModel():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(32, 32, 1)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(nClasses,  activation='softmax'))

    return model

In [ ]:
# Create the model
svhn_model = createModel()

In [ ]:
# Display the summary of the model
print("SVHN Model Architecture:")
svhn_model.summary()

In [ ]:
AdamOpt = Adam(learning_rate=0.001)
svhn_model.compile(optimizer=AdamOpt,
                   loss='sparse_categorical_crossentropy',
                   metrics=['sparse_categorical_accuracy'])

In [ ]:
batch_size = 256
epochs = 10

history = svhn_model.fit(x_train,
                         y_train,
                         batch_size=batch_size,
                         epochs=epochs,
                         verbose=1,
                         class_weight=class_weight_dict,
                         validation_data=(x_val, y_val))

In [ ]:
# Visualize training and validation errors
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Note we have started to over-fit (validation loss increases while training loss still decreases).

We can set the model to use early stopping based on increasing validation loss:

from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model with early stopping
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stopping])

In [ ]:
# Evaluate the model
test_loss, test_accuracy = svhn_model.evaluate(x_test, y_test)
print(f"\nSVHN Model - Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

In [ ]:
# Choose a few images for visualization
sample_images = x_test[:5]
sample_labels = y_test[:5]

In [ ]:
# Visualize predictions
predictions = svhn_model.predict(sample_images)
predicted_labels = [tf.argmax(prediction).numpy() for prediction in predictions]

In [ ]:
# Display the actual and predicted labels
print("\nActual Labels:", sample_labels)
print("Predicted Labels:", predicted_labels)

In [ ]:
# Visualize sample images
for i in range(len(sample_images)):
    plt.figure()
    plt.imshow(sample_images[i], cmap='gray')
    plt.title(f"Actual: {sample_labels[i]}, Predicted: {predicted_labels[i]}")
    plt.show()